In [12]:
%reload_ext autoreload 
%autoreload 2

In [13]:
# we can keep functions in the py file rather than here in the notebook
import function
function.imports()

Successfully imported functions.


#### We need to somehow get the emission polarization angle based on the IMU, GPS, and star camera data input streams. 